<a href="https://colab.research.google.com/github/sznajder/CSC2026/blob/main/JetTagging_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a Jet Tagging **DeepSets**

---
In this notebook, we perform a Jet identification task using a DeepSets multiclass classifier.
The problem consists in identifying a given jet as a quark, a gluon, a W, a Z, or a top.
We will represent each jet as a point cloud (or a "particle cloud") which consiststs of unordered set of jet's consistuent particles. Such a particle cloud representation of jets is efficient in incorporating raw information of jets and also explicitly respects the permutation symmetry.

Based on https://github.com/jngadiub/ML_course_Pavia_23

Modified by: Andre Sznajder

---

In [1]:
import os
import h5py
import glob, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Preparation of the training and validation samples

---
In order to import the dataset, we now
- clone the dataset repository (to import the data in Colab)
- load the h5 files in the data/ repository
- extract the data we need: a target and jetImage

To type shell commands, we start the command line with !

**nb, if you are running locally and you have already downloaded the datasets you can skip the cell below and, if needed, change the paths later to point to the folder with your previous download of the datasets.**

In [2]:
if not os.path.exists("Data-MLtutorial/JetDataset"):
    print("Dataset not found locally. Downloading and extracting...")
    os.system("curl -L https://cernbox.cern.ch/s/Hixs6KpgPFH8MN9/download -o Data-MLtutorial.tar.gz")
    os.system("tar -zxvf Data-MLtutorial.tar.gz")
    os.remove("Data-MLtutorial.tar.gz")
else:
    print("Dataset already exists. Skipping download.")

os.system("ls -al Data-MLtutorial/JetDataset/")

In [3]:
target = np.array([])
jetList = np.array([])
feat_names = dict()
# we cannot load all data on Colab. So we just take a few files
datafiles = ['Data-MLtutorial/JetDataset/jetImage_7_100p_30000_40000.h5',
             'Data-MLtutorial/JetDataset/jetImage_7_100p_60000_70000.h5',
             'Data-MLtutorial/JetDataset/jetImage_7_100p_50000_60000.h5',
             'Data-MLtutorial/JetDataset/jetImage_7_100p_10000_20000.h5',
             'Data-MLtutorial/JetDataset/jetImage_7_100p_0_10000.h5']
# if you are running locallt, you can use the full dataset doing
# for fileIN in glob.glob("tutorials/HiggsSchool/data/*h5"):
first=0
for fileIN in datafiles:
    print("Appending %s" %fileIN)
    f = h5py.File(fileIN)
    myJetList = np.array(f.get("jetConstituentList"))
    mytarget = np.array(f.get('jets')[0:,-6:-1])
    jetList = np.concatenate([jetList, myJetList], axis=0) if jetList.size else myJetList
    target = np.concatenate([target, mytarget], axis=0) if target.size else mytarget
    #save particles/nodes features names and their indecies in a dictionary
    if first==0:
      for feat_idx,feat_name in enumerate(list(f['particleFeatureNames'])[:-1]):
        feat_names[feat_name.decode("utf-8").replace('j1_','')] = feat_idx
      first=1
    del myJetList, mytarget
    f.close()
print(target.shape, jetList.shape)

Appending Data-MLtutorial/JetDataset/jetImage_7_100p_30000_40000.h5
Appending Data-MLtutorial/JetDataset/jetImage_7_100p_60000_70000.h5
Appending Data-MLtutorial/JetDataset/jetImage_7_100p_50000_60000.h5
Appending Data-MLtutorial/JetDataset/jetImage_7_100p_10000_20000.h5
Appending Data-MLtutorial/JetDataset/jetImage_7_100p_0_10000.h5
(50000, 5) (50000, 100, 16)


## One-Hot Encoding

The ground truth is incorporated in the ['j_g', 'j_q', 'j_w', 'j_z', 'j_t] vector of boolean, taking the form
*  [1, 0, 0, 0, 0] for gluons
*  [0, 1, 0, 0, 0] for quarks
*  [0, 0, 1, 0, 0] for W
*  [0, 0, 0, 1, 0] for Z
*  [0, 0, 0, 0, 1] for top quarks

This is what is called 'one-hot' encoding of a descrete label (typical of ground truth for classification problems). These labels are the 'target' for our classification tasks. Let's convert it back to single-column encoding :


In [4]:
num_classes = len(np.unique(target))
label_names= ["gluon", "quark", "W", "Z", "top"]

## Jet Constituents features

In [5]:
feat_to_consider = 'etarel,phirel,pt,e,ptrel,erel,deltaR'.split(',')
feat_idx = [feat_names[name] for name in feat_to_consider]
jetList = jetList[:,:,feat_idx]
print('Particles/Nodes considered features : ',feat_to_consider)

Particles/Nodes considered features :  ['etarel', 'phirel', 'pt', 'e', 'ptrel', 'erel', 'deltaR']


## Prepare the train and test datasets

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(jetList, target, test_size=0.33)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print('Jets shape : ',jetList.shape)
print('Target/Labels shape : ',target.shape)
del jetList, target

(33500, 100, 7) (16500, 100, 7) (33500, 5) (16500, 5)
Jets shape :  (50000, 100, 7)
Target/Labels shape :  (50000, 5)


In [7]:
# keras imports
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, Flatten
from keras.layers import GlobalAveragePooling1D, BatchNormalization, Activation
from keras.utils import plot_model
from keras import backend as K
from keras import metrics
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
print(f"TensorFlow {tf.__version__}")



TensorFlow 2.17.0


In [ ]:
#
# Attention implementation from Stefania Cristina:
# https://machinelearningmastery.mystagingwebsite.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras/
#
from tensorflow import math, matmul, reshape, shape, transpose, cast, float32
from keras.layers import Dense, Layer
from keras.backend import softmax


# Implementing the Scaled-Dot Product Attention
class DotProductAttention(Layer):
    def __init__(self, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)

    def call(self, queries, keys, values, d_k, mask=None):
        # Scoring the queries against the keys after transposing the latter, and scaling
        scores = matmul(queries, keys, transpose_b=True) / math.sqrt(cast(d_k, float32))

        # Apply mask to the attention scores
        if mask is not None:
            scores += -1e9 * mask

        # Computing the weights by a softmax operation
        weights = softmax(scores)

        # Computing the attention by a weighted sum of the value vectors
        return matmul(weights, values)

# Implementing the Multi-Head Attention
class MultiHeadAttention(Layer):
    def __init__(self, h, d_k, d_v, d_model, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.attention = DotProductAttention()  # Scaled dot product attention
        self.heads = h  # Number of attention heads to use
        self.d_k = d_k  # Dimensionality of the linearly projected queries and keys
        self.d_v = d_v  # Dimensionality of the linearly projected values
        self.d_model = d_model  # Dimensionality of the model
        self.W_q = Dense(d_k)  # Learned projection matrix for the queries
        self.W_k = Dense(d_k)  # Learned projection matrix for the keys
        self.W_v = Dense(d_v)  # Learned projection matrix for the values
        self.W_o = Dense(d_model)  # Learned projection matrix for the multi-head output

    def reshape_tensor(self, x, heads, flag):
        if flag:
            # Tensor shape after reshaping and transposing: (batch_size, heads, seq_length, -1)
            x = reshape(x, shape=(shape(x)[0], shape(x)[1], heads, -1))
            x = transpose(x, perm=(0, 2, 1, 3))
        else:
            # Reverting the reshaping and transposing operations: (batch_size, seq_length, d_k)
            x = transpose(x, perm=(0, 2, 1, 3))
            x = reshape(x, shape=(shape(x)[0], shape(x)[1], self.d_k))
        return x

    def call(self, queries, keys, values, mask=None):
        # Rearrange the queries to be able to compute all heads in parallel
        q_reshaped = self.reshape_tensor(self.W_q(queries), self.heads, True)
        # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)

        # Rearrange the keys to be able to compute all heads in parallel
        k_reshaped = self.reshape_tensor(self.W_k(keys), self.heads, True)
        # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)

        # Rearrange the values to be able to compute all heads in parallel
        v_reshaped = self.reshape_tensor(self.W_v(values), self.heads, True)
        # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)

        # Compute the multi-head attention output using the reshaped queries, keys and values
        o_reshaped = self.attention(q_reshaped, k_reshaped, v_reshaped, self.d_k, mask)
        # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)

        # Rearrange back the output into concatenated form
        output = self.reshape_tensor(o_reshaped, self.heads, False)
        # Resulting tensor shape: (batch_size, input_seq_length, d_v)

        # Apply one final linear projection to the output to generate the multi-head attention
        # Resulting tensor shape: (batch_size, input_seq_length, d_model)
        return self.W_o(output)

In [ ]:
#
# Attention implementation from Sebastian W.
#
from tensorflow import keras

class Attn(keras.layers.Layer):
    def __init__(self, dim = 16, nhead = 1, nbits = 8, **kwargs):
        super(Attn, self).__init__()
        
        self.dim = dim
#        self.dim_out = dim_out
        self.nhead = nhead
        self.nbits = nbits

        # Set QKeras quantizer 
        if nbits == 1:
            qbits = 'binary(alpha=1)'
        elif nbits == 2:
            qbits = 'ternary(alpha=1)'
        else:
            qbits = 'quantized_bits({},0,alpha=1)'.format(nbits)
            
        
        self.qD   = QDense(dim,  kernel_quantizer=qbits, bias_quantizer=qbits, name='attn_query')
        self.kD   = QDense(dim,  kernel_quantizer=qbits, bias_quantizer=qbits, name='attn_key')
        self.vD   = QDense(dim,  kernel_quantizer=qbits, bias_quantizer=qbits, name='attn_value')
        self.outD = QDense(dim,  kernel_quantizer=qbits, bias_quantizer=qbits, name='attn_out')

    def get_config(self):
        config = super().get_config()
        config.update({
            "dim": self.dim,
            "nhead": self.nhead,
            "nbits": self.nbits,
        })
        return config
        
    
    def call(self, input):
        input_shape = input.shape
        shape_ = (-1, input_shape[1], self.nhead, self.dim//self.nhead)
        perm_ = (0, 2, 1, 3)

        print ("Attention Input shape = ", input_shape)
        
        q = self.qD(input)
        q = tf.reshape(q, shape = shape_)
        q = tf.transpose(q, perm = perm_)

        print ("Attention Reshaped shape = ", q.shape)

        
#        k = self.qD(input) # Bug from Sebastian code
        k = self.kD(input)
        k = tf.reshape(k, shape = shape_)
        k = tf.transpose(k, perm = perm_)

#        v = self.qD(input) # Bug from Sebastian code
        v = self.vD(input)
        v = tf.reshape(v, shape = shape_)
        v = tf.transpose(v, perm = perm_)

        a = tf.matmul(q, k, transpose_b=True)
        a = tf.nn.softmax(a / q.shape[3]**0.5, axis = 3)

        out = tf.matmul(a, v)
        out = tf.transpose(out, perm = perm_)
        out = tf.reshape(out, shape = (-1, input_shape[1], self.dim))
        out = self.outD(out)

        print("Attention output shape = ",out.shape)
        
        return out

In [18]:
# My simplified Attention implementation
import tensorflow as tf
from tensorflow import math, matmul, reshape, shape, transpose, cast, float32
from keras.layers import Dense, Layer
from keras.ops import softmax

class Attn(keras.layers.Layer):
    def __init__(self, dim = 16, nhead = 1, **kwargs):
        super(Attn, self).__init__()
        
        self.dim = dim
#        self.dim_out = dim_out
        self.nhead = nhead

            
        self.vD   = Dense(dim, name='attn_value')
        self.outD = Dense(dim, name='attn_out')

    def get_config(self):
        config = super().get_config()
        config.update({ "dim": self.dim, "nhead": self.nhead })
        return config

    
    def call(self, input):
        input_shape  = input.shape
        print("input shape = ",input_shape)

        output_shape = (input_shape[0], input_shape[1], self.dim)
        print("output shape = ",output_shape)
        
#        shape_ = (-1, input_shape[1], self.nhead, self.dim//self.nhead)
        shape_ = (input_shape[0], input_shape[1], self.nhead, -1)
        perm_ = (0, 2, 1, 3)

        v = self.vD(input)
        v = tf.reshape(v, shape = shape_)
        v = tf.transpose(v, perm = perm_)

        a = tf.matmul(v, v, transpose_b=True)
        a = tf.nn.softmax(a / v.shape[3]**0.5, axis = 3)

        out = tf.matmul(a, v)
        out = tf.transpose(out, perm = perm_)
        out = tf.reshape(out, shape = output_shape)
        out = self.outD(out)
        
        return out


In [19]:
nnodes_phi = 32
nnodes_rho = 16
activ      = "relu"

# Input tensor shape
nconstit = X_train.shape[1]
nfeat = X_train.shape[2]


# Instantiate Tensorflow input tensors in Batch mode
inp = Input(shape=(nconstit,nfeat), name="inp")   # Conv1D input format


# Input point features BatchNormalization
h = BatchNormalization(name='BatchNorm')(inp)

# Attention
dim =24
nhead=8
h = Attn(dim,nhead)(h)

# Phi MLP ( permutation equivariant layers )
h = Dense(nnodes_phi)(h)
h = Activation(activ)(h)
h = Dense(nnodes_phi)(h)
h = Activation(activ)(h)
h = Dense(nnodes_phi)(h)
phi_out = Activation(activ)(h)


# Agregate features (taking mean) over set elements
mean = GlobalAveragePooling1D(name='avgpool_1')(phi_out)      # return mean of features over jets constituents

# Rho MLP
h = Dense(nnodes_rho)(mean)
h = Activation(activ)(h)
h = Dense(num_classes)(h)
out = Activation('softmax',name='softmax_g')(h)

# Build the model
model = Model(inputs=inp, outputs=out)

input shape =  (None, 100, 7)
output shape =  (None, 100, 24)
input shape =  (None, 100, 7)
output shape =  (None, 100, 24)


/Users/sznajder/WorkM1/miniforge3/envs/tf2_17/lib/python3.11/site-packages/keras/src/layers/layer.py:1381: UserWarning: Layer 'attn_5' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Failed to convert elements of (None, 100, 24) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.''
  warnings.warn(
/Users/sznajder/WorkM1/miniforge3/envs/tf2_17/lib/python3.11/site-packages/

TypeError: Exception encountered when calling Attn.call().

[1mCould not automatically infer the output shape / dtype of 'attn_5' (of type Attn). Either the `Attn.call()` method is incorrect, or you need to implement the `Attn.compute_output_spec() / compute_output_shape()` method. Error encountered:

Failed to convert elements of (None, 100, 24) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.[0m

Arguments received by Attn.call():
  • args=('<KerasTensor shape=(None, 100, 7), dtype=float32, sparse=False, name=keras_tensor_5>',)
  • kwargs=<class 'inspect._empty'>

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.CategoricalAccuracy()])
model.summary()

## Train the model

In [ ]:
batch_size = 128
epochs = 50

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
fig,axes = plt.subplots(2)
print(history.history.keys())
axes[0].plot(history.history["categorical_accuracy"])
axes[0].plot(history.history["val_categorical_accuracy"])
axes[0].set_title("Accuracy")
axes[0].legend(["Training", "Validation"])

axes[1].plot(history.history["loss"])
axes[1].plot(history.history["val_loss"])
axes[1].legend(["Training", "Validation"])
axes[1].set_title("Loss")

fig.show()



# Building the ROC Curves

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_curve, auc
predict_val = model.predict(X_val)
df = pd.DataFrame()
fpr = {}
tpr = {}
auc1 = {}

plt.figure()
for i, label in enumerate(label_names):

        df[label] = y_val[:,i]
        df[label + '_pred'] = predict_val[:,i]

        fpr[label], tpr[label], threshold = roc_curve(df[label],df[label+'_pred'])

        auc1[label] = auc(fpr[label], tpr[label])

        plt.plot(tpr[label],fpr[label],label='%s tagger, auc = %.1f%%'%(label,auc1[label]*100.))
plt.semilogy()
plt.xlabel("sig. efficiency")
plt.ylabel("bkg. mistag rate")
plt.ylim(0.000001,1)
plt.grid(True)
plt.legend(loc='lower right')
plt.show()